In [7]:
#import all library
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,balanced_accuracy_score,precision_recall_curve, roc_curve
from all_functions import *
from joblib import dump, load

"""
Select feature selection, Training, Features
"""
feature_selection='Strong_Feature'
Training='2016'
Features=9

#Define path
file_maturity_2016='C:/Users/tusha/Downloads/Peanut_Maturity.csv'
file_maturity_2017='C:/Users/tusha/Downloads/Peanut_Maturity_.csv'
feature_maturity_2016='C:/All/Peanut_Maturity_Classification'
feature_maturity_2017='C:/All/Peanut_Maturity_Classification_2017'


#Dataset Creation
Index_2016,Color_index_2016,All_columns,Maturity_2016,Final_Data_2016,Final_Data_Selected_2016=datacreate_2016(file=file_maturity_2016\
                                                                  ,path1=feature_maturity_2016,W_select=199)
Index_2017,Color_index_2017,Maturity_2017,Final_Data_2017,Final_Data_Selected_2017=datacreate_2017(file=file_maturity_2017\
                                                                  ,path2=feature_maturity_2017,W_select=199)

if Training=='2016':
    X_train_data,y_train_data,Train_Color_Index,Train_Index=Final_Data_Selected_2016,Maturity_2016,Color_index_2016,Index_2016
    X_test_data,y_test_data,Test_Color_Index,Test_Index=Final_Data_Selected_2017,Maturity_2017,Color_index_2017,Index_2017
elif Training=='2017':
    X_train_data,y_train_data,Train_Color_Index,Train_Index=Final_Data_Selected_2017,Maturity_2017,Color_index_2017,Index_2017
    X_test_data,y_test_data,Test_Color_Index,Test_Index=Final_Data_Selected_2016,Maturity_2016,Color_index_2016,Index_2016


if feature_selection=='Strong_Feature':
    X_train,X_test,y_train,y_test=Strong_feature_train_test_dataset_create(Features=Features,X_train=X_train_data,\
                                                        y_train=y_train_data,X_test=X_test_data,\
                                                        y_test=y_test_data)
elif feature_selection=='ANOVA':
    X_train,X_test,y_train,y_test=anova_train_test_dataset_create(Features=Features,X_train=X_train_data,\
                                                        y_train=y_train_data,X_test=X_test_data,\
                                                        y_test=y_test_data)    

#Performance Measures    
pipeline_rf = load(f'RF_Best_{Features}_{Training}_{feature_selection}.joblib') 
pipeline_rf.score(X_train, y_train),pipeline_rf.score(X_test, y_test)

y_pred_train=pipeline_rf.predict(X_train)
y_pred_test=pipeline_rf.predict(X_test)

Train_Confusion_matrix_rf, Test_Confusion_matrix_rf = confusion_matrix(y_train,y_pred_train), \
confusion_matrix(y_test,y_pred_test)
Train_specificity_rf, Test_specificity_rf = specificity(Train_Confusion_matrix_rf), specificity(Test_Confusion_matrix_rf)
Train_accuracy_score_rf,Test_accuracy_score_rf = pipeline_rf.score(X_train, y_train),pipeline_rf.score(X_test, y_test);
Train_precision_score_rf, Test_precision_score_rf=precision_score(y_train,y_pred_train,pos_label=0), \
precision_score(y_test,y_pred_test,pos_label=0)
Train_recall_score_rf, Test_recall_score_rf=recall_score(y_train,y_pred_train,pos_label=0), \
recall_score(y_test,y_pred_test,pos_label=0)
Train_balanced_accuracy_score_rf,Test_balanced_accuracy_score_rf=balanced_accuracy_score(y_train,y_pred_train),\
balanced_accuracy_score(y_test,y_pred_test)

Training_black_yellow=[pipeline_rf.score(X_train[Train_Color_Index[i]], y_train[Train_Color_Index[i]]) for i in range(4)]
Testing_black_yellow=[pipeline_rf.score(X_test[Test_Color_Index[i]], y_test[Test_Color_Index[i]]) for i in range(4)]
Training_species=[pipeline_rf.score(X_train[i*45:(i+1)*45], y_train[i*45:(i+1)*45]) for i in range(len(y_train)//45)]
Testing_species=[pipeline_rf.score(X_test[i*45:(i+1)*45], y_test[i*45:(i+1)*45]) for i in range(len(y_test)//45)]

file = open(f"RF_Best_{Features}_{Training}_{feature_selection}_final.txt", "w")
file.write(f"\nBest parameter for Random Forest Model is:\n {str(pipeline_rf)}")
file.write(f"\n Metrics: Train_result ,Test_result ")
file.write(f"\n Confusion_matrix: Train_result: {Train_Confusion_matrix_rf},Test_result: {Test_Confusion_matrix_rf}")
file.write(f"\n Accuracy_score: Train_result: {Train_accuracy_score_rf},Test_result: {Test_accuracy_score_rf}")
file.write(f"\n Precision_score: Train_result: {Train_precision_score_rf},Test_result: {Test_precision_score_rf}")
file.write(f"\n Recall_score: Train_result: {Train_recall_score_rf},Test_result: {Test_recall_score_rf}")
file.write(f"\n Specificity: Train_result: {Train_specificity_rf},Test_result: {Test_specificity_rf}")
file.write(f"\n Balanced_accuracy: Train_result: {Train_balanced_accuracy_score_rf},Test_result: {Test_balanced_accuracy_score_rf}")
file.write(f"\n Accuracy Score_Color_species: Train_result: {Training_species},Test_result: {Testing_species}")
file.write(f"\n Accuracy Score_Color (Black,Brown,Orange,Yellow): Train_result: {Training_black_yellow},Test_result: {Testing_black_yellow}")
file.close()